In [1]:
# CÉLULA 1: Importar bibliotecas e carregar artefactos

import pandas as pd
import numpy as np
import joblib
from tensorflow.keras.models import load_model
import os

# --- Carregar o Modelo e o Scaler ---
try:
    model = load_model('../models/oracle_lstm_v1.h5')
    scaler = joblib.load('../models/scaler_v1.pkl')
    print("Modelo e Scaler carregados com sucesso!")
except Exception as e:
    print(f"Erro ao carregar os ficheiros: {e}")

# Mostrar um resumo do modelo para confirmar que foi carregado corretamente
model.summary()


2025-09-20 15:00:08.818830: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M3
2025-09-20 15:00:08.818968: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 8.00 GB
2025-09-20 15:00:08.818971: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 2.67 GB
2025-09-20 15:00:08.819107: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-09-20 15:00:08.819115: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Modelo e Scaler carregados com sucesso!


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 60, 50)         │        12,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 60, 50)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 50)             │        20,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 50)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 3)              │           153 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 33,155 (129.52 KB)

 Trainable params: 33,153 (129.50 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)

In [7]:
# CÉLULA 2 (CORRIGIDA): Função para Engenharia de Features

def add_all_features(df):
    # --- Features de Indicadores Técnicos ---
    df['sma_20'] = df['close'].rolling(window=20).mean()
    df['ema_20'] = df['close'].ewm(span=20, adjust=False).mean()
    
    delta = df['close'].diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
    rs = gain / loss
    df['rsi_14'] = 100 - (100 / (1 + rs))
    
    df['bollinger_upper'] = df['sma_20'] + (df['close'].rolling(window=20).std() * 2)
    df['bollinger_lower'] = df['sma_20'] - (df['close'].rolling(window=20).std() * 2)
    
    ema_12 = df['close'].ewm(span=12, adjust=False).mean()
    ema_26 = df['close'].ewm(span=26, adjust=False).mean()
    df['macd'] = ema_12 - ema_26
    df['macd_signal'] = df['macd'].ewm(span=9, adjust=False).mean()

    # --- ADICIONAR A FEATURE EM FALTA ---
    df['daily_return'] = df['close'].pct_change()

    # --- Features de Tempo ---
    df['day_of_week'] = df.index.dayofweek.astype(float)
    df['day_of_month'] = df.index.day.astype(float)
    df['week_of_year'] = df.index.isocalendar().week.astype(float)
    df['month'] = df.index.month.astype(float)
    
    # Remover linhas com NaN
    df.dropna(inplace=True)
    
    return df

print("Função 'add_all_features' atualizada.")


Função 'add_all_features' atualizada.


In [9]:
# CÉLULA 3 (CORRIGIDA NOVAMENTE): Preparar os dados e fazer a previsão

# 1. Carregar os dados históricos do BTC
try:
    btc_df = pd.read_feather('../user_data/data/binance/BTC_USDT-1d.feather')
    btc_df['date'] = pd.to_datetime(btc_df['date'])
    btc_df.set_index('date', inplace=True)
    print("Dados históricos do BTC carregados.")
except Exception as e:
    print(f"Erro ao carregar dados históricos: {e}")

# 2. Aplicar a engenharia de features
df_features = add_all_features(btc_df.copy())

# 3. Pegar na última sequência de dados
TIME_STEPS = 60
last_sequence = df_features.tail(TIME_STEPS)

# 4. Separar as features (X) desta sequência
X_pred_raw = last_sequence.drop(columns=['open', 'high', 'low', 'close'])

# --- CORREÇÃO CRUCIAL: Garantir a ordem das colunas ---
# O scaler espera que as colunas estejam na mesma ordem do treino.
# Vamos obter a ordem correta a partir do próprio scaler.
feature_order = scaler.get_feature_names_out()
X_pred = X_pred_raw[feature_order]

# 5. Normalizar a sequência
X_pred_scaled = scaler.transform(X_pred)

# 6. Reformatar para o formato da LSTM
X_pred_seq = np.expand_dims(X_pred_scaled, axis=0)

# 7. Fazer a previsão!
prediction = model.predict(X_pred_seq)

# 8. Interpretar o resultado
print("\\n--- Previsão do Oráculo para o Próximo Dia ---")
print(f"Probabilidade de Venda (-1): {prediction[0][0]*100:.2f}%")
print(f"Probabilidade Neutra (0):   {prediction[0][1]*100:.2f}%")
print(f"Probabilidade de Compra (1):  {prediction[0][2]*100:.2f}%")


Dados históricos do BTC carregados.


2025-09-20 15:09:00.097138: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
\n--- Previsão do Oráculo para o Próximo Dia ---
Probabilidade de Venda (-1): 87.21%
Probabilidade Neutra (0):   0.02%
Probabilidade de Compra (1):  12.77%
